In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# data = pd.read_csv(r"https://raw.githubusercontent.com/avnyadav/sensor-fault-detection/main/aps_failure_training_set1.csv",na_values="na")
# data.shape

(36188, 171)

In [ ]:
# data.to_csv("../data/sensor_info.csv",index=False,header=True)

In [5]:
df = pd.read_csv('../data/sensor_info.csv')
df.shape

(36188, 171)

In [6]:
from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36188 entries, 0 to 36187
Columns: 171 entries, class to eg_000
dtypes: float64(169), int64(1), object(1)
memory usage: 47.2+ MB


In [8]:
numerical_columns = [col for col in df.columns if df[col].dtype !="O"]
categorical_columns = [col for col in df.columns if df[col].dtype =="O"]

print(f"numerical columns length is {len(numerical_columns)} and columns is {numerical_columns}")
print(f"categorical columns length is {len(categorical_columns)} and columns is {categorical_columns}")

numerical columns length is 170 and columns is ['aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000', 'ag_001', 'ag_002', 'ag_003', 'ag_004', 'ag_005', 'ag_006', 'ag_007', 'ag_008', 'ag_009', 'ah_000', 'ai_000', 'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000', 'ao_000', 'ap_000', 'aq_000', 'ar_000', 'as_000', 'at_000', 'au_000', 'av_000', 'ax_000', 'ay_000', 'ay_001', 'ay_002', 'ay_003', 'ay_004', 'ay_005', 'ay_006', 'ay_007', 'ay_008', 'ay_009', 'az_000', 'az_001', 'az_002', 'az_003', 'az_004', 'az_005', 'az_006', 'az_007', 'az_008', 'az_009', 'ba_000', 'ba_001', 'ba_002', 'ba_003', 'ba_004', 'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009', 'bb_000', 'bc_000', 'bd_000', 'be_000', 'bf_000', 'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000', 'bl_000', 'bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000', 'bs_000', 'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000', 'cb_000', 'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000', 'ci_000', 'cj_

In [9]:
x = df.drop(columns=['class'],axis=1)
y = df['class']

In [12]:
y.value_counts()

class
neg    35188
pos     1000
Name: count, dtype: int64

In [11]:
num_columns = [col for col in x.columns if x[col].dtype !="O"]
cat_columns = [col for col in x.columns if x[col].dtype =="O"]

In [14]:
from imblearn.under_sampling import RandomUnderSampler

rs = RandomUnderSampler()
x_resample , y_resample = rs.fit_resample(x,y)
x_resample.shape, y_resample.shape

((2000, 170), (2000,))

In [17]:
X_train,X_test,y_train,y_test = train_test_split(x_resample,y_resample,test_size=0.2,random_state=42)

from sklearn.impute import SimpleImputer
num_pipe = Pipeline(steps=[
    ('impute',SimpleImputer(strategy='median')),
    ('scaler',MinMaxScaler())
])
cal_pipe = Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('encode',OneHotEncoder(handle_unknown='ignore',drop='first',sparse_output=False))
])

preprossesor = ColumnTransformer([
    ('num_col',num_pipe,num_columns),
    ('cat_col',cal_pipe,cat_columns)
],remainder='passthrough')

preprossesor.fit(X_train)

X_train = preprossesor.transform(X_train)
X_test = preprossesor.transform(X_test)

In [21]:
def model_ecaluation(actual,predicted):
    acc = accuracy_score(actual,predicted)
    cr = classification_report(actual,predicted)
    
    print(f"accuracy_score is {acc}")
    print(f"classification_report is {cr}")

In [22]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

model_ecaluation(actual=y_test,predicted=y_pred)

accuracy_score is 0.95
classification_report is               precision    recall  f1-score   support

         neg       0.98      0.91      0.95       199
         pos       0.92      0.99      0.95       201

    accuracy                           0.95       400
   macro avg       0.95      0.95      0.95       400
weighted avg       0.95      0.95      0.95       400



In [23]:
def total_cost(a,p):
    tn,fp,fn,tp = confusion_matrix(a,p).ravel()
    cost = 10*fp + 500*fn
    
    return cost

In [24]:
total_cost(a=y_test,p=y_pred)

np.int64(1670)